In [255]:
import pandas as pd

In [256]:
from datasets import load_dataset

train_data, test_data = load_dataset("yelp_review_full" , split =['train[0:800]', 'test[0:200]'])

Reusing dataset yelp_review_full (C:\Users\SANDY\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)


  0%|          | 0/2 [00:00<?, ?it/s]

In [257]:
print(type(train_data))

<class 'datasets.arrow_dataset.Dataset'>


In [258]:
df = train_data.to_pandas()
df1 = test_data.to_pandas()

In [259]:
df.head(10)


,label,text
0,4,dr. goldberg offers everything i look for in a...
1,1,"Unfortunately, the frustration of being Dr. Go..."
2,3,Been going to Dr. Goldberg for over 10 years. ...
3,3,Got a letter in the mail last week that said D...
4,0,I don't know what Dr. Goldberg was like before...
5,4,Top notch doctor in a top notch practice. Can'...
6,4,Dr. Eric Goldberg is a fantastic doctor who ha...
7,0,I'm writing this review to give you a heads up...
8,1,Wing sauce is like water. Pretty much a lot of...
9,2,Decent range somewhat close to the city. The ...


In [260]:
x_train = df['text']
y_train = df['label'] #value of y is dependent on X

x_test = df1['text']
y_test = df1['label']

y_train.shape

(800,)

In [261]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=1000, lower=True)
tokenizer.fit_on_texts(x_train)

x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

vocab_size = len(tokenizer.word_index) + 1

In [262]:
print(x_train[1])
print(y_train[1])

[751, 1, 6, 196, 657, 9, 4, 6, 1, 146, 78, 25, 19, 28, 170, 80, 11, 31, 390, 116, 8, 297, 13, 218, 116, 613, 90, 1, 377, 8, 263, 691, 139, 291, 6, 5, 37, 50, 107, 79, 51, 10, 13, 42, 986, 5, 454, 19, 8, 3, 21, 421, 150, 16, 391, 19, 170, 80, 2, 3, 36, 67, 37, 8, 18, 21, 614, 873, 18, 21, 19, 716, 243, 367, 511, 1, 377, 47, 2, 23, 199, 1, 47, 19, 13, 3, 254, 13, 3, 21, 5, 173, 657, 139, 255]
1


In [263]:
from keras.preprocessing.sequence import pad_sequences

#padding makes sure that every sequence will have the same line, missing space is filled by zero

maxlen = 100

x_train = pad_sequences(x_train, padding = 'post',maxlen=maxlen)
x_test = pad_sequences(x_test, padding = 'post',maxlen=maxlen)

print(x_train[3, :])

[ 97   4  11   1 187 404  13 118 657   9 922   5   5 133   4 175  32  11
  72  70  29  49  96  12  91 125   4 175  11  13  18 200  38 368 307  29
  33 615  33 347   5 111   4 831   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [264]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, dtype ="uint8")

print(y_train.shape)
print(y_train[0])

(800, 5)
[0 0 0 0 1]


In [268]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, SimpleRNN
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
import numpy as np

#It is converted as 3D input as specified

x_train = np.array(x_train).reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = np.array(x_test).reshape(x_test.shape[0], x_test.shape[1], 1)

def vanilla_run(layer_name):
    model = Sequential()
    model.add(layer_name(100, input_shape=(100,1), return_sequences=False))
    model.add(Dense(5))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
    model.summary()
    return model

In [269]:
model = vanilla_run(SimpleRNN)
history=model.fit(x_train, y_train, epochs=50, batch_size=50)

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_22 (SimpleRNN)   (None, 100)               10200     
                                                                 
 dense_18 (Dense)            (None, 5)                 505       
                                                                 
 activation_20 (Activation)  (None, 5)                 0         
                                                                 
Total params: 10,705
Trainable params: 10,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
16/16 [==============================] - 1s 11ms/step - loss: 1.6358 - accuracy: 0.2087
Epoch 2/50
16/16 [==============================] - 0s 10ms/step - loss: 1.5653 - accuracy: 0.2738
Epoch 3/50
16/16 [==============================] - 0s 11ms/step - loss: 1.5519 - accuracy: 0.2875
Epoch 4/50
16/16

In [273]:
a = ["I hate their service"]
a = tokenizer.texts_to_sequences(a)
a= np.array(a)
a = pad_sequences(a, padding='post', maxlen=maxlen)
a = np.array(a).reshape(a.shape[0], a.shape[1], 1)

prediction = model.predict(a)
print(prediction[0])
print(len(prediction[0])-1 -np.argmax(prediction))

# 0-4 is the scale. max number represents satisfaction

[0.20864727 0.20412067 0.18072188 0.12203641 0.28447372]
0
